In [1]:
# this cell enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

In [2]:
# import all your modules here
import json
import pandas as pd

import config_v2 as cfg
from library_ubidots_v2 import Ubidots as ubi

In [3]:
# set your constants here
baseline=cfg.BASELINE
study=cfg.STUDY

In [4]:
print(baseline)
print(study)

['2022-11-01', '2023-02-28']
['2023-03-01', '2023-03-31']


In [5]:
df_devices = ubi.get_available_devices_v2(label='bancolombia', level='group', page_size=1000)

In [6]:
df_vars = ubi.get_available_variables(list(df_devices['device_id']))

In [7]:
df_vars = df_vars[df_vars['variable_label'].isin(cfg.WHITELISTED_VAR_LABELS)]
VAR_IDS_TO_REQUEST = list(df_vars['variable_id'])
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [8]:
CHUNK_SIZE = 1
DATE_INTERVAL_REQUEST = {'start': baseline[0], 'end': study[1]}

df = None
lst_responses = []
n_vars = len(VAR_IDS_TO_REQUEST)
print(f"Making request for the following interval: Baseline:{baseline}, Study:{study}")
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    chunk = VAR_IDS_TO_REQUEST[idx_start:idx_end]

    

    response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST, 
    )

    current_idx = idx_end+1
    
    if (current_idx > n_vars):
        current_idx = n_vars

    print(f"Progress: {100*(current_idx)/n_vars:0.1f}%")

    if response.status_code != 204:

        lst_responses.append(response)
    else: 
        print(f"Empty response for chunk {idx}")

df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)


Making request for the following interval: Baseline:['2022-11-01', '2023-02-28'], Study:['2023-03-01', '2023-03-31']
Progress: 0.6%
Progress: 0.9%
Progress: 1.2%
Progress: 1.6%
Progress: 1.9%
Progress: 2.2%
Progress: 2.5%
Progress: 2.8%
Progress: 3.1%
Progress: 3.4%
Progress: 3.8%
Progress: 4.1%
Progress: 4.4%
Progress: 4.7%
Progress: 5.0%
Progress: 5.3%
Progress: 5.6%
Progress: 5.9%
Progress: 6.2%
Progress: 6.6%
Progress: 6.9%
Progress: 7.2%
Progress: 7.5%
Progress: 7.8%
Progress: 8.1%
Progress: 8.4%
Progress: 8.8%
Progress: 9.1%
Progress: 9.4%
Progress: 9.7%
Progress: 10.0%
Progress: 10.3%
Progress: 10.6%
Progress: 10.9%
Progress: 11.2%
Progress: 11.6%
Progress: 11.9%
Progress: 12.2%
Progress: 12.5%
Progress: 12.8%
Progress: 13.1%
Progress: 13.4%
Progress: 13.8%
Progress: 14.1%
Progress: 14.4%
Progress: 14.7%
Progress: 15.0%
Progress: 15.3%
Progress: 15.6%
Progress: 15.9%
Progress: 16.2%
Progress: 16.6%
Progress: 16.9%
Progress: 17.2%
Progress: 17.5%
Progress: 17.8%
Progress: 18.1%
P

In [9]:
pd.to_pickle(df, project_path / 'data'/ _PICKLED_DATA_FILENAME)